In [1]:
import pandas as pd
import numpy as np
import pycmap
import os.path
from datetime import datetime, timedelta
import pytz

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
# calls api from cmap and uses my token to do that
api = pycmap.API(token = "d24b5240-ea3a-45bd-8174-fdb7830d805f")

In [3]:
## this is a way to call the seaflow metadata to python from google drive
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1Tsi7OWIZWfCQJqLDpId2aG_i-8Cp-p63PYjjvDkOtH4'
SAMPLE_RANGE_NAME = 'metadata'
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
try:
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')

except HttpError as err:
    print(err)

In [4]:
seaflow_meta=pd.DataFrame(values)
seaflow_meta.columns=seaflow_meta.iloc[0]
seaflow_meta1 = seaflow_meta.iloc[1:]
seaflow_meta
# this seaflow_meta 

,cruise,Cruise ID,Project,R2R doi,Location,Year,Month,Day Of Year,Instrument,Ship,...,Discrete samples,Beads,Metadata,Filtration,Gating,Curation,Zenodo,Database size (Gb),References,Comments
0,cruise,Cruise ID,Project,R2R doi,Location,Year,Month,Day Of Year,Instrument,Ship,...,Discrete samples,Beads,Metadata,Filtration,Gating,Curation,Zenodo,Database size (Gb),References,Comments
1,CMOP_1,W0903A,CMOP,10.7284/900616,Columbia River,2009,May,,740,R/V Wecoma,...,,\n,,,,,,,,"this has no sds.tab file, will need to get data"
2,Thompson_0,TN243,H2O,10.7284/904001,Puget Sound,2009,November,,740,R/V Thomas G Thompson,...,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,,Ribalet et al. 2011,"Little/no Pro. ""Croco"" is probably cryptophytes."
3,CMOP_3,W1005A,CMOP,10.7284/904064,Columbia River,2010,May,141 - 147,740,R/V Wecoma,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,,,LAT/LON missing for nearly 400 points; no PAR;...
4,Thompson_1,TN248,-,10.7284/904006,Gulf of Alaska,2010,May,,740,R/V Thomas G Thompson,...,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,0.4,Pavlesky et al. 2013; Hyrkas et al. 2016,NO DATA IN R2R FOR THIS CRUISE; some lat missi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,HOT343,,HOT-343,,Aloha,2023,August,None,None,None,...,None,None,None,None,None,None,None,None,None,None
120,HOT344,,HOT-344,,Aloha,2023,September,None,None,None,...,None,None,None,None,None,None,None,None,None,None
121,HOT345,,HOT-345,,Aloha,2023,October,None,None,None,...,None,None,None,None,None,None,None,None,None,None
122,HOT346,,HOT-346,,Aloha,2023,November,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [5]:
def read_sfl(x):
    df = pd.read_csv(x, sep = '\t')
    cruise_split = x.rsplit('/',1)[-1]
    #output will be a string
    cruise_short = cruise_split.replace('.sfl', '')
    cruise_list = cruise_short.split('_')
    if len(cruise_list) > 2:
        cruise = "_".join([cruise_list[0], cruise_list[1]])
    elif len(cruise_list) == 2:
        cruise = cruise_list[0]
    ship = seaflow_meta.loc[seaflow_meta['cruise'] ==  cruise, 'Ship'].values[0]
    print(ship)
    df['cruise'] = cruise
    df['ship'] = ship
    return df

#### .values converts pd.series to list but we had to get string from the list which is the first one 

In [6]:
sfl = read_sfl("https://raw.githubusercontent.com/seaflow-uw/seaflow-sfl/master/curated/TN397_130_130.sfl")

R/V Thomas G Thompson


In [7]:
sfl

,FILE,DATE,FILE DURATION,LAT,LON,CONDUCTIVITY,SALINITY,OCEAN TEMP,PAR,BULK RED,STREAM PRESSURE,EVENT RATE,cruise,ship
0,2021_323/2021-11-19T03-01-27+00-00,2021-11-19T03:01:27+00:00,180.073,32.706600,-117.236300,NaN,NaN,NaN,0.029,115.7579,0.75,0,TN397_130,R/V Thomas G Thompson
1,2021_323/2021-11-19T03-04-27+00-00,2021-11-19T03:04:27+00:00,180.080,32.706600,-117.236300,NaN,NaN,NaN,0.029,91.6572,0.75,0,TN397_130,R/V Thomas G Thompson
2,2021_323/2021-11-19T03-07-28+00-00,2021-11-19T03:07:28+00:00,180.035,32.706600,-117.236300,NaN,NaN,NaN,0.028,93.5062,0.75,0,TN397_130,R/V Thomas G Thompson
3,2021_323/2021-11-19T03-10-28+00-00,2021-11-19T03:10:28+00:00,27.204,32.706600,-117.236300,NaN,NaN,NaN,0.029,93.5062,0.75,0,TN397_130,R/V Thomas G Thompson
4,2021_323/2021-11-19T03-11-26+00-00,2021-11-19T03:11:26+00:00,180.040,32.706600,-117.236300,NaN,NaN,NaN,0.025,138.7690,0.75,0,TN397_130,R/V Thomas G Thompson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11038,2021_348/2021-12-14T22-16-28+00-00,2021-12-14T22:16:28+00:00,180.125,19.192050,-156.297450,5.420927,34.818908,26.324992,2468.471,74.5972,2.34,0,TN397_130,R/V Thomas G Thompson
11039,2021_348/2021-12-14T22-19-28+00-00,2021-12-14T22:19:28+00:00,162.503,19.200191,-156.303327,5.419084,34.815336,26.312100,2474.775,79.6548,2.34,0,TN397_130,R/V Thomas G Thompson
11040,2021_348/2021-12-14T22-35-49+00-00,2021-12-14T22:35:49+00:00,180.096,19.247258,-156.337150,5.423963,34.837700,26.329175,1878.722,203.7869,2.34,0,TN397_130,R/V Thomas G Thompson
11041,2021_348/2021-12-14T22-38-49+00-00,2021-12-14T22:38:49+00:00,180.108,19.255750,-156.343075,5.424074,34.836658,26.331650,1780.900,203.7869,2.34,0,TN397_130,R/V Thomas G Thompson


In [39]:
pd.to_datetime(sfl['DATE'])
# this converts sfl date column from object type to datetime in UTC

datetime_index = pd.DatetimeIndex(sfl.DATE.values).tz_convert('UTC')
df2 = sfl.set_index(datetime_index).reset_index()
df2.rename(columns={"index": "Date"}, inplace = True)
df2.drop(columns = ['DATE'], inplace = True)

#### trying to convert datetime64 to datetimeindex for groupby function to work

In [42]:
sfl_hourly = df2.groupby([pd.Grouper(key = 'Date', freq = 'H'), 'cruise']) # its not hourly, its still every 3 minutes WAH 
sfl_hourly.head()
# goal here is to group cruise and date column and . agg mean par to get hourly par data

,Date,FILE,FILE DURATION,LAT,LON,CONDUCTIVITY,SALINITY,OCEAN TEMP,PAR,BULK RED,STREAM PRESSURE,EVENT RATE,cruise,ship
0,2021-11-19 03:01:27+00:00,2021_323/2021-11-19T03-01-27+00-00,180.073,32.706600,-117.236300,NaN,NaN,NaN,0.029,115.7579,0.75,0,TN397_130,R/V Thomas G Thompson
1,2021-11-19 03:04:27+00:00,2021_323/2021-11-19T03-04-27+00-00,180.080,32.706600,-117.236300,NaN,NaN,NaN,0.029,91.6572,0.75,0,TN397_130,R/V Thomas G Thompson
2,2021-11-19 03:07:28+00:00,2021_323/2021-11-19T03-07-28+00-00,180.035,32.706600,-117.236300,NaN,NaN,NaN,0.028,93.5062,0.75,0,TN397_130,R/V Thomas G Thompson
3,2021-11-19 03:10:28+00:00,2021_323/2021-11-19T03-10-28+00-00,27.204,32.706600,-117.236300,NaN,NaN,NaN,0.029,93.5062,0.75,0,TN397_130,R/V Thomas G Thompson
4,2021-11-19 03:11:26+00:00,2021_323/2021-11-19T03-11-26+00-00,180.040,32.706600,-117.236300,NaN,NaN,NaN,0.025,138.7690,0.75,0,TN397_130,R/V Thomas G Thompson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11033,2021-12-14 22:01:27+00:00,2021_348/2021-12-14T22-01-27+00-00,180.043,19.150017,-156.265983,5.425830,34.834900,26.350708,2222.102,78.0623,2.34,0,TN397_130,R/V Thomas G Thompson
11034,2021-12-14 22:04:27+00:00,2021_348/2021-12-14T22-04-27+00-00,180.010,19.158350,-156.272375,5.426306,34.835917,26.353900,2580.031,78.0623,2.34,0,TN397_130,R/V Thomas G Thompson
11035,2021-12-14 22:07:27+00:00,2021_348/2021-12-14T22-07-27+00-00,180.021,19.166775,-156.278808,5.426351,34.835758,26.354550,1850.836,78.0623,2.34,0,TN397_130,R/V Thomas G Thompson
11036,2021-12-14 22:10:27+00:00,2021_348/2021-12-14T22-10-27+00-00,180.006,19.175250,-156.285100,5.426361,34.838100,26.351542,2595.287,78.0623,2.34,0,TN397_130,R/V Thomas G Thompson
